<a href="https://colab.research.google.com/github/crow-intelligence/gettingStartedWithColab/blob/main/03_Using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 3
## Using Pytorch

### Load data
Load count vectorized data made at Milestone 1. Transform them into torch tensors.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pickle

import numpy as np
import torch

# load countvectorized data
X_train, X_test, y_train, y_test = pickle.load(open("/content/drive/My Drive/Colab Notebooks/01_data/vectorized_data.p", "rb"))

# numeric labels
def relabel(label):
  if label == "spam":
    return 0
  else:
    return 1

y_train = np.asarray([relabel(e) for e in y_train])
y_test = np.asarray([relabel(e) for e in y_test])

# convert them into tensors
X_train_tensor = torch.from_numpy(X_train.todense()).float()
X_test_tensor = torch.from_numpy(X_test.todense()).float()
Y_train_tensor = torch.from_numpy(np.array(y_train))
Y_test_tensor = torch.from_numpy(np.array(y_test))

### Make Dataset

In [ ]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

train_data = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

test_data = TensorDataset(X_test_tensor, Y_test_tensor)
test_loader = DataLoader(test_data,batch_size=16, shuffle=True)


### Define the network

In [ ]:
class Network(torch.nn.Module):
  def __init__(self,vocab_size,hidden_units,num_classes): 
    super().__init__()
    #First fully connected layer
    self.fc1 = torch.nn.Linear(vocab_size,hidden_units)
    #Second fully connected layer
    self.fc2 = torch.nn.Linear(hidden_units,num_classes)
    #Final output of sigmoid function      
    self.output = torch.nn.Sigmoid()

  def forward(self,x):
    fc1 = self.fc1(x)
    fc2 = self.fc2(fc1)
    output = self.output(fc2)
    return output[:, -1]


### Training

In [ ]:
NUM_EPOCHS = 200
VOCAB_SIZE = 8110
HIDDEN_UNITS = 5
OUT_CLASSES = 1
LEARNING_RATE = 0.001

# save checkpoints


#Initialize model
model = Network(VOCAB_SIZE,HIDDEN_UNITS,OUT_CLASSES)
model.cuda()
print(model)
#Initialize optimizer
optimizer =torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
#Initialize loss function
loss_fun = torch.nn.BCELoss()
for i in range(NUM_EPOCHS):
  # print(i)
  PATH = f"/content/drive/My Drive/Colab Notebooks/01_models/pytorch/pytorch_model_epoch{i}.pt"
  for x_batch, y_batch in train_loader:
    model.train()
    y_pred = model(x_batch.cuda())
    loss = loss_fun(y_pred.cuda(), y_batch.float().cuda())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  torch.save({"epoch": i,
              "model_state_dict": model.state_dict(),
              "optimizer_state_dict": optimizer.state_dict(),
              "loss": loss}, PATH)
print('After {} epoch training loss is {}'.format(i,loss.item()))


Network(
  (fc1): Linear(in_features=8110, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=1, bias=True)
  (output): Sigmoid()
)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
After 199 epoch training loss is 0.0015496076084673405


In [ ]:
from sklearn.metrics import classification_report

y_pred_tensor = model(X_test_tensor.cuda())
y_pred_probs = y_pred_tensor.cpu().detach().numpy()
# we want high recall & precision for spam
y_pred = np.asarray([(lambda x: 1 if x > 0.80 else 0) (x) for x in y_pred_probs])

print(classification_report(y_test, y_pred, target_names=["spam", "ham"]))


              precision    recall  f1-score   support

        spam       0.90      0.92      0.91       191
         ham       0.99      0.98      0.99      1203

    accuracy                           0.97      1394
   macro avg       0.94      0.95      0.95      1394
weighted avg       0.98      0.97      0.98      1394

